<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/pyenv-virtualenv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00


In [4]:
!pip install insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 111.7 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055395 sha256=8dfdea2451a40367b4dd620ff806ce124a4e8733eeba3b1923dc08199c29c598
  Stored in directory: /root/.cache/pip/wheels/e3/d0/80/e3773fb8b6d1cca87ea1d33d9b1f20a223a6493c896da249b5
Successfully built insightface


In [5]:
!pip install moviepy

In [2]:
import cv2
import numpy as np
from tqdm import tqdm
from insightface.app import FaceAnalysis
from moviepy.editor import VideoFileClip, AudioFileClip

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
video_name = "zoom_480p.mov"
video_path = "./" + video_name
mask_path  = '.'.join(video_path.split('.')[:-1]) + '_mask.mp4'
audio_path = '.'.join(video_path.split('.')[:-1]) + '.wav'
output_path = '.'.join(video_path.split('.')[:-1]) + '_audio_mask.mp4'
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path, ffmpeg_params=["-ac", "1"])  #モノラルで出力

MoviePy - Writing audio in ./zoom_480p.wav


MoviePy - Done.


In [5]:
class INSIGHTFACE(object):
    def __init__(self, ctx_id=0, det_size_x=640, det_size_y=640):
        self.app = FaceAnalysis(root='./.insightface')
        self.app.prepare(ctx_id=ctx_id, det_size=(det_size_x, det_size_y))
        self.faces = []

    def get_InsightFace(self, img):
        self.faces = self.app.get(np.asarray(img))
        #for face in faces:
        #    kps   = face.kps.astype(np.int)
        #    print(kps)
        rimg  = self.app.draw_on(img, self.faces)
        #cv2.imwrite("./output.jpg", rimg)
        return img, rimg, self.faces

    def draw_mesen(self, img):
        KPS = []
        for face in self.faces:
            kps = face.kps.astype(np.int)
            KPS.append(kps)
        EYE = _EYE = [[]] * 2
        out_img = img
        for _eye in KPS:
            EYE[0] = _eye[0]
            EYE[1] = _eye[1]
            lineSize  = abs(np.linalg.norm(np.array(EYE[0])-np.array(EYE[1])))
            lineWidth = int(lineSize / 4)
            if lineWidth <= 10:
                lineWidth = 10
            lineAngle = 1
            if EYE[1][0]-EYE[0][0] != 0:
                lineAngle   = (EYE[1][1]-EYE[0][1])/(EYE[1][0]-EYE[0][0])
            _EYE[0][0] = EYE[0][0] - (lineSize / 3)
            _EYE[0][1] = lineAngle * (_EYE[0][0] - EYE[0][0]) + EYE[0][1]
            _EYE[1][0] = EYE[1][0] + (lineSize / 3)
            _EYE[1][1] = lineAngle * (_EYE[1][0] - EYE[0][0]) + EYE[0][1]
            out_img = cv2.line(out_img,
                              (_EYE[0][0], _EYE[0][1]),
                              (_EYE[1][0], _EYE[1][1]),
                              (0,0,0),
                              thickness=lineWidth)
        return out_img

In [6]:
IF = INSIGHTFACE()

  warnings.warn(



Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: ./.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [15]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 104.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.17.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.1.4.231227 requires numpy>=1.26.0; python_version < "3.13", but you have numpy 1.23.5 which is incompatible.
xarray 2024.9.0 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.


In [1]:
!cp ./drive/MyDrive/zoom_480p.mov ./

In [7]:
cap = cv2.VideoCapture(video_path)

In [8]:
if cap.isOpened() is False:
    cap = None

In [9]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = float(cap.get(cv2.CAP_PROP_FPS))
frameNum = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [10]:
codec = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(mask_path, codec, fps, (width, height), True)

In [11]:
bar = tqdm(total=frameNum, dynamic_ncols=True)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
ret, img = cap.read()
print(ret)
f_num = 0
while ret :
    f_num += 1
    input_img, draw_img, faces = IF.get_InsightFace(img)
    #if len(faces):
    out_img = IF.draw_mesen(input_img)
    #input_img, draw_img, faces = IF.get_InsightFace(input_img)  #なぜか、2回処理すると取りこぼしがなくなる。
    #out_img = IF.draw_mesen(out_img)
    writer.write(out_img)
    ret, img = cap.read()
    bar.update(1)
writer.release()

  0%|          | 0/38882 [00:00<?, ?it/s]

True


To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kps = face.kps.astype(np.int)

100%|█████████▉| 38881/38882 [3:46:20<00:00,  3.96it/s]

In [13]:
!cp ./zoom_480p_mask.mp4 ./drive/MyDrive

In [14]:
video_clip = VideoFileClip(mask_path)
audio_clip = AudioFileClip(audio_path)

In [15]:
video_clip = video_clip.set_audio(audio_clip)
video_clip.write_videofile(output_path)

100%|█████████▉| 38881/38882 [3:47:31<00:00,  3.96it/s]

Moviepy - Building video ./zoom_480p_audio_mask.mp4.
MoviePy - Writing audio in zoom_480p_audio_maskTEMP_MPY_wvf_snd.mp3



100%|█████████▉| 38881/38882 [3:47:48<00:00,  3.96it/s]

MoviePy - Done.
Moviepy - Writing video ./zoom_480p_audio_mask.mp4




100%|█████████▉| 38881/38882 [3:49:27<00:00,  3.96it/s]

Moviepy - Done !
Moviepy - video ready ./zoom_480p_audio_mask.mp4


In [17]:
! ls -alh

total 869M
drwxr-xr-x 1 root root 4.0K Oct 18 16:52 .
drwxr-xr-x 1 root root 4.0K Oct 18 12:45 ..
drwxr-xr-x 4 root root 4.0K Sep 20 13:21 .config
drwx------ 6 root root 4.0K Oct 18 12:58 drive
drwxr-xr-x 3 root root 4.0K Oct 18 13:00 .insightface
drwxr-xr-x 1 root root 4.0K Sep 20 13:22 sample_data
-rw-r--r-- 1 root root  94M Oct 18 16:52 zoom_480p_audio_mask.mp4
-rw-r--r-- 1 root root 255M Oct 18 16:49 zoom_480p_mask.mp4
-rw------- 1 root root 390M Oct 18 12:58 zoom_480p.mov
-rw-r--r-- 1 root root 131M Oct 18 13:02 zoom_480p.wav


In [18]:
!cp ./zoom_480p_audio_mask.mp4 ./drive/MyDrive